In [11]:
spark.stop()
sc.stop()

NameError: name 'spark' is not defined

In [12]:
import findspark
findspark.init()

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import sys
import string
from csv import reader
from functools import reduce
from pyspark.sql import functions as f
from collections import defaultdict
import datetime
%load_ext autoreload
%autoreload 2
sys.path.insert(0, '../src')
from tableCollections import TableCollections

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
spark = SparkSession \
                .builder \
                .appName("TableCollections") \
                .config("spark.io.compression.codec", "snappy") \
                .config("spark.shuffle.service.enabled", "false") \
                .config("spark.dynamicAllocation.enabled", "false") \
                .config("spark.rdd.compress", "true") \
                .getOrCreate()
sc = spark.sparkContext

## Query: ReturnIntersecWithinCols()

In [23]:
t1 = spark.read.format('csv').options(header='true',inferschema='true').load('/user/jh5990/open_data_csv/qgea-i56i.csv')

In [24]:
parkingTable = spark.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/parking-violations-header.csv')
openTable = spark.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/open-violations-header.csv')

In [7]:
tc = TableCollections(spark, sc)
tc.register(openTable, "open")
tc.register(parkingTable, "parking")
columns = ["parking^plate_id", "open^plate"]
tc.intersecWithinCols(columns).show()

num metadata file exists for table open
string metadata file exists for table open
num metadata file exists for table parking
timestamp metadata file exists for table parking
string metadata file exists for table parking
+---------+-----+
|col_value|count|
+---------+-----+
|  GWR6544|    2|
|  GMZ6430|    2|
|  GTZ3429|    2|
|  GWH8612|    2|
|  GKJ3196|    2|
|  HCT2162|    2|
|  JKX3874|    2|
|  JDA2966|    2|
| MAGUJ0DE|    2|
|  HAX7497|    2|
| T682962C|    2|
|  GFC7118|    2|
|  GUX8085|    2|
|  GYD6262|    2|
|  GMB9512|    2|
|  GZC3304|    2|
|  HFA1602|    2|
|  HBU4498|    2|
|  HCN8863|    2|
|  GXN8364|    2|
+---------+-----+
only showing top 20 rows



## Query: frequentVals()
Input: list of columns (possibly from multiple tables), integer topN  
Output: topN rows from union of the columns with topN frequency

In [30]:
tc = TableCollections(spark, sc)
tc.register(t1, "TableOne")
# tc.register(parkingTable, "parking")
columns = ["TableOne^PREM_TYP_DESC"]
tc.frequentVals(columns, 10).show()

num metadata file exists for table TableOne
string metadata file exists for table TableOne
+--------------------+-------+-------------+
|           col_value|    cnt|     col_name|
+--------------------+-------+-------------+
|              STREET|1841239|PREM_TYP_DESC|
|RESIDENCE - APT. ...|1158315|PREM_TYP_DESC|
|     RESIDENCE-HOUSE| 549568|PREM_TYP_DESC|
|RESIDENCE - PUBLI...| 419936|PREM_TYP_DESC|
|               OTHER| 148410|PREM_TYP_DESC|
| COMMERCIAL BUILDING| 144019|PREM_TYP_DESC|
|TRANSIT - NYC SUBWAY| 117859|PREM_TYP_DESC|
|    DEPARTMENT STORE| 107343|PREM_TYP_DESC|
|         CHAIN STORE| 107217|PREM_TYP_DESC|
|       PUBLIC SCHOOL|  71369|PREM_TYP_DESC|
+--------------------+-------+-------------+



## Query: getCardinality()

In [25]:
t2 = spark.read.format('csv').options(header='true',inferschema='true').load('/user/jh5990/open_data_csv/kpav-sd4t.csv')
t3 = spark.read.format('csv').options(header='true',inferschema='true').load('/user/jh5990/open_data_csv/ay9k-vznm.csv')

In [27]:
tc = TableCollections(spark, sc)
tc.register(t2, "TableTwo")
tc.register(t3, "TableThree")
columns = ["TableTwo^Agency", "TableThree^SubSectorName"]
tc.getCardinality(columns).show()

num metadata file exists for table TableTwo
string metadata file exists for table TableTwo
num metadata file exists for table TableThree
string metadata file exists for table TableThree
+-------------+-----+
|     col_name|count|
+-------------+-----+
|SubSectorName|   28|
|       Agency|   45|
+-------------+-----+



## Query: colsWithAndWithout()

### EX1: Is Brooklyn misclassified as a city?
An empty result means that no columns misclassified Brooklyn as a city

In [16]:
t4 = spark.read.format('csv').options(header='true',inferschema='true').load('/user/jh5990/open_data_csv/swpk-hqdp.csv')
t5 = spark.read.format('csv').options(header='true',inferschema='true').load('/user/jh5990/open_data_csv/xi7c-iiu2.csv')
t6 = spark.read.format('csv').options(header='true',inferschema='true').load('/user/jh5990/open_data_csv/qcdj-rwhu.csv')

In [17]:
tc = TableCollections(spark, sc)
tc.register(t4, "TableFour")
tc.register(t5, "TableFive")
tc.register(t6, "TableSix")
columns = ["TableFour^Borough", "TableFive^Borough", "TableSix^CITY"]

tc.colsWithAndWithout(columns, ["BROOKLYN", "NEW YORK"], ["MANHATTAN"])

num metadata file exists for table TableFour
string metadata file exists for table TableFour
num metadata file exists for table TableFive
string metadata file exists for table TableFive
num metadata file exists for table TableSix
string metadata file exists for table TableSix
There are no columns that satisfies the condition


### EX2: Is this column cuisine type or nationality?

An empty result means that the column is for cuisine type

In [16]:
t7 = spark.read.format('csv').options(header='true',inferschema='true').load('/user/jh5990/open_data_csv/43nn-pn8j.csv')

In [17]:
tc = TableCollections(spark, sc)
tc.register(t7, "TableSeven")

columns = ["TableSeven^CUISINE DESCRIPTION"]

tc.colsWithAndWithout(columns, ["American", "Italian"], ["Bakery", "Donuts"])

There are no columns that satisfies the condition


## Query: returnOutliers()

In [26]:
columns = ["TableOne^PREM_TYP_DESC", "TableSeven^CUISINE DESCRIPTION"]
tc.returnOutliers(columns, 0.02)

Total number of data: 5580035
Maximum number of outliers to fetch: 111601
Actual number of outliers fetched: 110659
Number of rows in result dataframe: 32
Total number of data: 372810
Maximum number of outliers to fetch: 7456
Actual number of outliers fetched: 7127
Number of rows in result dataframe: 34
+--------------------+----+-------------+
|           col_value| cnt|     col_name|
+--------------------+----+-------------+
|             TRAMWAY| 106|PREM_TYP_DESC|
|        LOAN COMPANY| 404|PREM_TYP_DESC|
|            CEMETERY| 697|PREM_TYP_DESC|
|              MOSQUE| 757|PREM_TYP_DESC|
|          PHOTO/COPY| 780|PREM_TYP_DESC|
|OTHER HOUSE OF WO...| 823|PREM_TYP_DESC|
|TAXI/LIVERY (UNLI...| 957|PREM_TYP_DESC|
|FERRY/FERRY TERMINAL|1566|PREM_TYP_DESC|
|           BOOK/CARD|1784|PREM_TYP_DESC|
|         VIDEO STORE|2020|PREM_TYP_DESC|
|           SYNAGOGUE|2055|PREM_TYP_DESC|
|  ABANDONED BUILDING|2126|PREM_TYP_DESC|
|         MARINA/PIER|2366|PREM_TYP_DESC|
|         BUS (OTHER)|2